<a href="https://colab.research.google.com/github/albertomariapepe/Using-a-GT-for-PSP-via-GA-modelling/blob/main/Generating_Cost_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/pygae/clifford.git@master
!pip install biopython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#store cost plots as training features as shown in PDNET

from scipy.spatial.transform import Rotation as R
import numpy as np
from clifford.g3c import *
from clifford.tools.g3c import *
from clifford.tools.g3c.rotor_parameterisation import *
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from keras import backend as K
from math import acos
import tensorflow as tf
from matplotlib import pyplot as plt
import keras
from math import e, pi, cos
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.DSSP import DSSP
import os
import itertools  

import warnings
warnings.filterwarnings('ignore')

structure_id = "chain"
parser = PDBParser(PERMISSIVE=1)
dir1 = '/content/drive/MyDrive/chains-cameo/'
dir2 = '/content/drive/MyDrive/chains-test/'
dir3 = '/content/drive/MyDrive/chains/'

CONTATORE = 0
lst1 = os.listdir(dir1)
lst2 = os.listdir(dir2)
lst3 = os.listdir(dir3)


lst = [x for x in itertools.chain(lst1, lst2, lst3)]
lst = os.listdir('drive/MyDrive/Output-ALL/')
#lst.remove('.DS_Store')
lst.sort()
structure_id = "chain"
parser = PDBParser(PERMISSIVE=1)
lst = lst
for filename in lst:

    filename = os.path.splitext(filename)[0]
    filename = filename[:-3]
    filename = filename + '.pdb'
    print(filename)

    N = 0
    m = 0
    I = e1*e2*e3
    lamb = 1
    #counting the number of atoms
    idx = 0
    TOL =  15 #MEASURE IN ANGSTROM DEFINING THE RADIUS OF INTERACTION BETWEEN TWO RESIDUES
    chain_n = 0
    #structure = parser.get_structure(structure_id, dir + filename+'.pdb')
    if filename in lst1:
        structure = parser.get_structure(structure_id, dir1 + filename)
    if filename in lst2:
        structure = parser.get_structure(structure_id, dir2 + filename)
    if filename in lst3:
        structure = parser.get_structure(structure_id, dir3 + filename)

    cost_type = 'Eide'

    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MSE': 'M', 'MET': 'M'}

    #counting the number of chains
    for model in structure:
        for chain in model:
             for residue in chain:
                 for atom in residue:
                     if atom.altloc == "B":
                         del atom

    seq = []
    chain_id = []
    string_array = [] 
    import os.path
    chain_name = filename
    filename = filename
    CONTATORE = CONTATORE + 1
    print(filename, ": processing file n.", CONTATORE, "/", len(lst))
                    
    #storing 3D coordinates in an array P for each of the atoms
               
        
    i = 0
    m = 0

    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.resname in d3to1:
                    seq.append(d3to1[residue.resname]) 
                for atom in residue:
                    N = N + 1
                    if atom.name == "CA":   #counting the number of Calphas
                     idx = idx + 1
            string = ""
            string_array = np.append(string_array, string.join(seq))
            seq = []
    
    P = np.zeros([N, 3])    
    cnt = np.zeros([idx, 1])

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    P[m] = atom.get_coord()
                    m = m + 1
                    if atom.name == "CA":
                     cnt[i,0] = m 
                     i = i + 1 
    
    b = np.zeros([N-1, 3])
    
    #calculating bond lengths between atoms
    for i in range(0,N-1):   
        b[i] = P[i+1]-P[i]
   
    m = 0

    if cost_type == 'Eide':
      C_rot = np.zeros([idx, idx])
      C_tran = np.zeros([idx, idx])
      Cost = np.zeros([idx,idx])

    if cost_type == 'Tingelstadt':
      DIS = np.zeros([idx, idx])
      COS = np.zeros([idx, idx])
      Cost = np.zeros([idx,idx])

    if cost_type == 'Circle':
      Cost = np.zeros([idx,idx])
    
    if cost_type == 'Contact':
      Cost = np.zeros([idx,idx])

    for m in range(0, idx):
        for n in range(0, m):
          i = int(cnt[m,0])-1
          j = int(cnt[n,0])-1
    
          a = P[i-1,0]*e1 + P[i-1,1]*e2 + P[i-1,2]*e3   #coordinates of N
          b = P[i,0]*e1 + P[i,1]*e2 + P[i,2]*e3         #coordinates of Calpha
          c = P[i+1,0]*e1 + P[i+1,1]*e2 + P[i+1,2]*e3   #coordinates of C
    
          #subtract the centre of mass, so that C_tran does not increase along the chain
    
          '''
          M = (a + b + c)/3 
          a = a - M        
          b = b - M
          c = c - M
          '''
        
          d = P[j-1,0]*e1 + P[j-1,1]*e2 + P[j-1,2]*e3
          e = P[j,0]*e1 + P[j,1]*e2 + P[j,2]*e3 
          f = P[j+1,0]*e1 + P[j+1,1]*e2 + P[j+1,2]*e3 

          ''''
          d = d - M
          e = e - M
          f = f - M  
          '''

          if (abs(n - m) < int(idx/10)):
          if abs(b-e) < TOL:
            A = up(a)
            B = up(b)
            C = up(c)
            
            D = up(d)
            E = up(e)
            F = up(f)
                            
            
            P1 = A ^ B ^ C ^ einf  #first peptide plane
            P2 = D ^ E ^ F ^ einf  #successive peptide plane
            
            P1 = P1/abs(P1)
            P2 = P2/abs(P2)

            if cost_type == 'Contact':
                Cost[m,n] = 1
                            

            if cost_type == 'Eide':
            
              R = rotor_between_objects(P2, P1)  #rotor R in conformal

              C_rot_err = ((R - 1)*(~R - 1))(0) #contribution due to translation
              R_par = R|e4
              C_tran_err =  (R_par*~R_par)(0) #contribution due to rotation
            
              C_rot[m,n] = float(C_rot_err) 
              #print(C_rot_err)
              C_tran[m,n] = float(C_tran_err)

            if cost_type == 'Tingelstadt':

              costheta = (P1*P2)(0)  # equivalent to the inner product P1 | P2
              if costheta == -1.0:
                costheta = -1      #to avoid a "math domain error"
            
              theta = acos(costheta)
              costhetahalf = cos(theta/2)

              G = 0.5*P2/P1
            
              R = rotor_between_objects(P2, P1)  #rotor R in conformal
              T = rotor_between_objects(G, R)  #rotor R in conformal
            
              M = T*R*~T

              w = -2*(1 - T)|eo
              d = abs(w)
           
              DIS[m,n] = d
              COS[m,n] = (1 - costhetahalf)

            if cost_type == 'Circle':
              C1 = A ^ B ^ C
              C2 = D ^ E ^ F
            
              M = rotor_between_objects(C1,C2)
            
              x = abs((down(M*A*~M) - down(D))**2)
              y = abs((down(M*B*~M) - down(E))**2)
              z = abs((down(M*C*~M) - down(F))**2)

              Cost[m,n] = x + y + z
                              
    

    lamb = 1
    #Cost = lamb*C_rot + C_tran  
    Cost = C_rot

    Cost = Cost + Cost.T - np.diag(np.diag(Cost))

    '''
    plt.figure()
    plt.imshow(Cost) #cmap = bwr
    ticks = np.linspace(0,idx,1)
    plt.colorbar()
    plt.axis('off')
    '''

    train_data = []
    train_data.append(int(idx))
    train_data.append(string_array[0])
    train_data.append(Cost)

    #print(train_data)
    filename = os.path.splitext(filename)[0]
    
    a = np.load('/content/drive/MyDrive/Output-ALL/' + filename + '-cb.npy', allow_pickle= True)
    if (a[0] != int(idx)):
        print('error!!!')
    
    print(filename + '...done!')